## base model下载以及环境配置

In [ ]:
! pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('msmarco-distilbert-base-tas-b',cache_folder='sbert/')

In [ ]:
! pip install pandas
! pip install matplotlib
! pip install openpyxl

## 含query log的数据集构造

### 1. 从pkl文件中加载query数据

In [1]:
import pickle
import pandas as pd
import json
file_folder = 'bert-sql/pkl/'

datasets ="synthea"
## 读入pkl文件
with open(file_folder+"omop.pkl", "rb") as f:
    query1 = pickle.load(f)

with open(file_folder+"{}.pkl".format(datasets), "rb") as f:
    query2 = pickle.load(f)

print(len(query1))
print(len(query2))

267
111


### 2. 合并schema数据和query数据

In [3]:
data_filefolder = f"datasets/{datasets}/"
columns = len(query2)

csvFilePath = data_filefolder+"omop_{}_data.xlsx".format(datasets)
jsonFilePath = data_filefolder+"omop_{}.json".format(datasets)

df = pd.read_excel(csvFilePath)

jsonString = df.to_json(orient="records", indent=4)

## 将csv格式转成json格式
with open(jsonFilePath, "w") as jsonFile:
    jsonFile.write(jsonString)

## 写入sql查询信息
with open(jsonFilePath) as jsonFile:
    datas = json.load(jsonFile)


for i in range(len(datas)):
    idA = i // columns
    idB = i % columns
    
    del datas[i]["d1"]
    del datas[i]["d2"]
    del datas[i]["d3"]
    del datas[i]["d4"]
    try:
        datas[i]["queryA"] = query1[idA]
        datas[i]["queryB"] = query2[idB]
    except:
        print(i)
        print(idA,idB)
        break

with open(data_filefolder+"final.json".format(datasets), "w") as jsonFile:
    jsonString = json.dumps(datas, indent=4)
    jsonFile.write(jsonString)


### 划分train/val/test

In [4]:
## random split
import random 
import json

jsonFilePath = data_filefolder + "/final.json"

with open(jsonFilePath) as jsonFile:
    datas = json.load(jsonFile)

def data_split(full_list, ratio, shuffle=False):
    """
    数据集拆分: 将列表full_list按比例ratio（随机）划分为2个子列表sublist_1与sublist_2
    :param full_list: 数据列表
    :param ratio:     子列表1
    :param shuffle:   子列表2
    :return:
    """
    n_total = len(full_list)
    offset = int(n_total * ratio)
    if n_total == 0 or offset < 1:
        return [], full_list
    if shuffle:
        random.shuffle(full_list)
    sublist_1 = full_list[:offset]
    sublist_2 = full_list[offset:]
    return sublist_1, sublist_2

random.seed(5199)
test, others = data_split(datas, ratio=0.1, shuffle=True)
val,train = data_split(others, ratio=0.1/0.9, shuffle=True)

with open(data_filefolder + "/test0.json", "w") as jsonFile:
    jsonString = json.dumps(test, indent=4)
    jsonFile.write(jsonString)

with open(data_filefolder + "/val0.json", "w") as jsonFile:
    jsonString = json.dumps(val, indent=4)
    jsonFile.write(jsonString)

with open(data_filefolder + "/train0.json", "w") as jsonFile:
    jsonString = json.dumps(train, indent=4)
    jsonFile.write(jsonString)